In [ ]:
import sys

# to allow imports from outside the tsp_ml/datasets/ package
sys.path.insert(0, "/home/pimenta/tsp_ml/tsp_ml")

from paths import (
    TSP_TEST_DATASET_FOLDER_PATH
)
import csv
import torch
import networkx as nx
from torch_geometric.utils.convert import to_networkx

In [ ]:
# open csv and recover predictions
predictions = {}
filepath ="/home/pimenta/tsp_ml/data/tsp/predictions/test/predictions.csv" 
with open(filepath) as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        # print(row)
        id, pred, truth = row
        if id not in predictions.keys():
            predictions[id] = {"predictions": [], "truth": []}
        predictions[id]["predictions"].append(int(pred))
        predictions[id]["truth"].append(int(truth))
        # print(truth)
print(predictions)


In [ ]:
# load tsp intance
# id = "0d3f3ac84455fd38dc01215d0248514a"
id = "5a88f6b12ee690fc8bccc13cb312abdb"
filepath = TSP_TEST_DATASET_FOLDER_PATH / f"tsp_instance_{id}.pt"
tsp_graph = torch.load(filepath)


In [ ]:
# cast to nx.Graph
nx_tsp_graph = to_networkx(
    data=tsp_graph,
    node_attrs=["node_features"],
    edge_attrs=["y", "distance"],
)
nx_tsp_graph

In [ ]:
# add predictions to graph

# create Dict: edge_tuple -> pred
edge_tuples = nx_tsp_graph.edges()
predictions_list = predictions[id]["predictions"]
pyg_truth_values = predictions[id]["truth"]
pred_attr_dict = {}
for edge_tuple, pred in zip(edge_tuples, predictions_list):
    pred_attr_dict[edge_tuple] = pred

# for DEBUG
# pyg_truth_attr_dict = {}
# for edge_tuple, pyg_truth in zip(edge_tuples, pyg_truth_values):
#     pyg_truth_attr_dict[edge_tuple] = pyg_truth

nx.set_edge_attributes(G=nx_tsp_graph, values=pred_attr_dict, name="prediction")
# nx.set_edge_attributes(G=nx_tsp_graph, values=pyg_truth_attr_dict, name="pyg_truth")

# cast to undirected graph: nx.Graph
nx_tsp_graph = nx_tsp_graph.to_undirected()
nx_tsp_graph

In [ ]:
# plot graph with optimal_route and predictions

# get node_positions
node_positions = {}
for node_id, node_features in nx_tsp_graph.nodes(data=True):
    node_positions[node_id] = node_features["node_features"]

# get optimal and predicted routes
optimal_route_edges = []
predicted_route_edges = []
for (edge_src, edge_dst, edge_features) in nx_tsp_graph.edges(data=True):
    if edge_features['y'] == 1:
        optimal_route_edges.append((edge_src, edge_dst))
    if edge_features['prediction'] == 1:
        predicted_route_edges.append((edge_src, edge_dst))
    
# plot graph
nx.draw(G=nx_tsp_graph, pos=node_positions)
# plot optimal route in green
nx.draw_networkx_edges(G=nx_tsp_graph,pos=node_positions,edgelist=optimal_route_edges, edge_color ='g', width=12)
# plot predicted route in red
nx.draw_networkx_edges(G=nx_tsp_graph,pos=node_positions,edgelist=predicted_route_edges, edge_color ='r', width=3)